In [26]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np

from memory_fix import reduce_mem_usage

In [3]:
transactions_train = reduce_mem_usage( pd.read_csv('data/transactions_train.csv') )

Memory usage of dataframe is 668.09 MB
Memory usage after optimization is: 250.53 MB
Decreased by 62.5%


In [7]:
group_text = pd.read_excel('data/small_group_description.xlsx')

In [9]:
from sklearn.cluster import SpectralClustering
import scipy.sparse as sps


In [ ]:
def get_diff(array):
    new_array = list()
    for i in range(1, len(array)):
        new_array.append(array[i] - array[i-1])
    
    return np.mean(new_array), np.max(new_array), np.min(new_array), np.std(new_array)

In [113]:
from tqdm import tqdm

In [129]:
def get_target_stat(transaction, target):
    stat = {'mean_date':[], 'max_date':[],'min_date':[],'std_date':[], 'mean_sum':[], 'mean_count':[]}
    
    small = transaction[transactions_train.small_group == target]
    group = small.groupby(['client_dk', 'trans_date'])["amount"].agg(['count', 'sum'])
    
    for ind in tqdm(transaction['client_dk'].unique()):
        try:
            smb = group.loc[ind]
            mean, mx, mn, std = get_diff(list(smb.index))

            stat['mean_date'].append(mean)
            stat['max_date'].append(mx)
            stat['min_date'].append(mn)
            stat['std_date'].append(std)

            stat['mean_sum'].append(np.mean(smb['sum']))
            stat['mean_count'].append(np.mean(smb['count']))
        except Exception:
            stat['mean_date'].append(-10000)
            stat['max_date'].append(-10000)
            stat['min_date'].append(-10000)
            stat['std_date'].append(-10000)

            stat['mean_sum'].append(-10000)
            stat['mean_count'].append(-10000)
            
        
    return stat

In [130]:
ds = get_target_stat(transactions_train, target=27)

100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:14<00:00, 1679.18it/s]


In [132]:
pd.DataFrame(ds)

,mean_date,max_date,min_date,std_date,mean_sum,mean_count
0,19.894737,161,1,34.643895,118.249405,1.15
1,-10000.000000,-10000,-10000,-10000.000000,-10000.000000,-10000.00
2,9.333333,14,5,3.681787,29.453499,1.00
3,111.000000,244,27,95.124480,172.415497,1.75
4,17.071429,68,1,19.721143,40.917667,1.40
...,...,...,...,...,...,...
24995,-10000.000000,-10000,-10000,-10000.000000,-10000.000000,-10000.00
24996,-10000.000000,-10000,-10000,-10000.000000,-10000.000000,-10000.00
24997,-10000.000000,-10000,-10000,-10000.000000,-10000.000000,-10000.00
24998,7.666667,14,3,4.642796,12.192714,1.00
